## Tworzenie grafu z geodataframe

In [1]:
import srai
import pandas as pd
import geopandas as gpd
from srai.regionalizers import H3Regionalizer, geocode_to_region_gdf
from srai.embedders import CountEmbedder
from srai.joiners import IntersectionJoiner
from srai.loaders import OSMOnlineLoader

In [2]:
area = geocode_to_region_gdf("Wrocław, Poland")

In [3]:
area.head()

,geometry
region_id,
"Lower Silesian Voivodeship, Poland, Wrocław","POLYGON ((16.80734 51.13895, 16.80859 51.13887..."


In [50]:
regionalizer = H3Regionalizer(resolution=8) # https://h3geo.org/docs/core-library/restable/
regions = regionalizer.transform(area)
regions

/home/gry-mar/osm-proj/gradient/env/lib/python3.8/site-packages/h3ronpy/arrow/vector.py:147: DeprecationWarning: The all_intersecting parameter is deprecated and will be removed. Use containment_mode instead.
  return vector.wkb_to_cells(


,geometry
region_id,
881e20476bfffff,"POLYGON ((17.06902 51.14580, 17.06660 51.14152..."
881e204059fffff,"POLYGON ((16.95106 51.14480, 16.94865 51.14052..."
881e20420dfffff,"POLYGON ((16.90522 51.17367, 16.90280 51.16939..."
881e204319fffff,"POLYGON ((16.88084 51.14264, 16.87843 51.13836..."
881e20475bfffff,"POLYGON ((17.12136 51.16499, 17.11894 51.16072..."
...,...
881e2041c3fffff,"POLYGON ((16.94997 51.08165, 16.94756 51.07737..."
881e204249fffff,"POLYGON ((16.89716 51.19611, 16.89475 51.19184..."
881e20415dfffff,"POLYGON ((16.87176 51.10197, 16.86935 51.09769..."


In [54]:
print(regions.iloc[0]['geometry'])

POLYGON ((17.06902348673659 51.14580118662367, 17.06660198564568 51.141524048141, 17.07168961994108 51.1383087166747, 17.079198797447713 51.13937034004275, 17.081621061626862 51.14364740694358, 17.076533385295708 51.146862922077226, 17.06902348673659 51.14580118662367))


In [55]:
print(regions.iloc[1]['geometry'])

POLYGON ((16.95106179158528 51.14479635598944, 16.94864817780492 51.140517282359625, 16.953739837067456 51.13730743507905, 16.961245156680384 51.13837647795663, 16.96365953577112 51.142655481192385, 16.95856783002262 51.145865511963756, 16.95106179158528 51.14479635598944))


In [56]:
type(regions.iloc[1]['geometry'])

shapely.geometry.polygon.Polygon

In [59]:
print(regions.iloc[1]['geometry'].boundary)

LINESTRING (16.95106179158528 51.14479635598944, 16.94864817780492 51.140517282359625, 16.953739837067456 51.13730743507905, 16.961245156680384 51.13837647795663, 16.96365953577112 51.142655481192385, 16.95856783002262 51.145865511963756, 16.95106179158528 51.14479635598944)


In [9]:
# gdf = gpd.read_file("./data/wypadki/2023_szczegoly_zdarzen_dr.shx")
wypadki = gpd.read_file("./data/wypadki/2023_szczegoly_zdarzen_dr.shx")
wypadki = wypadki.to_crs('EPSG:4326')
wypadki.head()
     

,POJ_SUMA,POJ_ROWER,POJ_TRAM,POJ_BUS,UCZ_SUMA,UCZ_PIESI,Z_SUMA,Z_PIESI,Z_IN,CR_SUMA,...,RDZ_DR,RDZ_NAW,STAN_NAW,SYGN,OZN_POZ,OSW,WAR_ATM,USZK_P_POJ,ROK,geometry
0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,dwie jezdnie jednokierunkowe,twarda,sucha,"jest, działa",jest,światło dzienne,dobre warunki atmosferyczne,NaN,2023,POINT (16.96645 51.13083)
1,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,jednojezdniowa dwukierunkowa,twarda,sucha,brak,jest,światło dzienne,dobre warunki atmosferyczne,NaN,2023,POINT (16.96015 51.12240)
2,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,dwie jezdnie jednokierunkowe,twarda,sucha,"jest, działa",jest,światło dzienne,dobre warunki atmosferyczne,NaN,2023,POINT (17.03003 51.13736)
3,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,dwie jezdnie jednokierunkowe,twarda,sucha,"jest, działa",jest,światło dzienne,dobre warunki atmosferyczne,NaN,2023,POINT (17.03003 51.13736)
4,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,dwie jezdnie jednokierunkowe,twarda,sucha,"jest, działa",jest,"świt, zmrok",dobre warunki atmosferyczne,NaN,2023,POINT (17.03003 51.13736)


In [18]:
from srai.plotting import plot_regions

query = {"amenity": "bicycle_parking"}
area = geocode_to_region_gdf("Wrocław, Poland")
loader = OSMOnlineLoader()

parks_gdf = loader.load(area, query)
folium_map = plot_regions(area, colormap=["rgba(0,0,0,0)"], tiles_style="CartoDB positron")
parks_gdf.explore(m=folium_map, color="forestgreen")

In [19]:
parks_gdf

,geometry,amenity
feature_id,,
node/436769494,POINT (17.08337 51.14051),bicycle_parking
node/469129015,POINT (17.00702 51.07465),bicycle_parking
node/469129016,POINT (17.00682 51.07507),bicycle_parking
node/702379163,POINT (17.00824 51.09533),bicycle_parking
node/968841019,POINT (17.02792 51.11651),bicycle_parking
...,...,...
way/1196126710,"POLYGON ((17.09546 51.10907, 17.09553 51.10910...",bicycle_parking
way/1203450306,"POLYGON ((16.99630 51.09969, 16.99656 51.09970...",bicycle_parking
way/1208367906,"POLYGON ((17.06744 51.06102, 17.06746 51.06107...",bicycle_parking


In [20]:
query1 = {"amenity": "vending_machine"}
area = geocode_to_region_gdf("Wrocław, Poland")
loader = OSMOnlineLoader()

vending_gdf = loader.load(area, query1)
folium_map = plot_regions(area, colormap=["rgba(0,0,0,0)"], tiles_style="CartoDB positron")
vending_gdf.explore(m=folium_map, color="forestgreen")

In [21]:
vending_gdf

,geometry,amenity
feature_id,,
node/265643966,POINT (17.01233 51.12131),vending_machine
node/369029509,POINT (17.04909 51.08267),vending_machine
node/1213165684,POINT (17.02240 51.11177),vending_machine
node/1213165714,POINT (17.02074 51.11218),vending_machine
node/1737437042,POINT (17.03605 51.09726),vending_machine
...,...,...
node/11196092617,POINT (17.03763 51.07332),vending_machine
node/11197980434,POINT (17.03626 51.09756),vending_machine
node/11197980435,POINT (17.03628 51.09869),vending_machine


In [23]:
appended_gdf = pd.concat([parks_gdf, vending_gdf], ignore_index=True)

In [24]:
type(appended_gdf)

geopandas.geodataframe.GeoDataFrame

In [25]:
appended_gdf

,geometry,amenity
0,POINT (17.08337 51.14051),bicycle_parking
1,POINT (17.00702 51.07465),bicycle_parking
2,POINT (17.00682 51.07507),bicycle_parking
3,POINT (17.00824 51.09533),bicycle_parking
4,POINT (17.02792 51.11651),bicycle_parking
...,...,...
1470,POINT (17.03763 51.07332),vending_machine
1471,POINT (17.03626 51.09756),vending_machine
1472,POINT (17.03628 51.09869),vending_machine
1473,POINT (17.03627 51.09869),vending_machine


In [26]:
from srai.constants import WGS84_CRS, FEATURES_INDEX

In [28]:
query = {'amenity': appended_gdf['amenity'], 'geometry': appended_gdf['geometry']}
features_x = gpd.GeoDataFrame(
    query,
    crs=WGS84_CRS,
    index=gpd.pd.Index(name=FEATURES_INDEX, data=[i for i in range(len(appended_gdf))]),
)
regions = regionalizer.transform(area)
joiner = IntersectionJoiner()
joint = joiner.transform(regions, features_x, return_geom=True)

/home/gry-mar/osm-proj/gradient/env/lib/python3.8/site-packages/h3ronpy/arrow/vector.py:147: DeprecationWarning: The all_intersecting parameter is deprecated and will be removed. Use containment_mode instead.
  return vector.wkb_to_cells(


In [34]:
joint

geometry
region_id       feature_id                                                   
881e2040c5fffff 1233        LINESTRING (16.99987 51.11291, 17.00020 51.11285)
                1234        LINESTRING (16.99982 51.11273, 17.00013 51.11268)
                1314        LINESTRING (17.00483 51.11441, 17.00495 51.11437)
881e204019fffff 1238        LINESTRING (16.97707 51.12533, 16.97737 51.12524)
                1239        LINESTRING (16.97712 51.12541, 16.97721 51.12539)
...                                                                       ...
881e20462dfffff 1333        POLYGON ((17.13858 51.16227, 17.13861 51.16227...
881e204763fffff 1334        POLYGON ((17.07844 51.13245, 17.07873 51.13240...
881e204549fffff 1335        POLYGON ((17.09553 51.10910, 17.09555 51.10908...
881e204e43fffff 1338        POLYGON ((17.04285 51.06467, 17.04285 51.06475...
                1339        POLYGON ((17.04285 51.06448, 17.04285 51.06454...

[1473 rows x 1 columns]

In [36]:

joint.explore(
    m=folium_map,
    style_kwds=dict(color="darkgreen", opacity=1, fillColor="darkgreen", fillOpacity=1),
    marker_kwds=dict(radius=2),
)

In [43]:
wide_embedder = CountEmbedder(count_subcategories=True)
wide_embedding = wide_embedder.transform(regions, features_x, joint)
wide_embedding.head()

,amenity_bicycle_parking,amenity_vending_machine
region_id,,
881e20476bfffff,2,0
881e204059fffff,2,0
881e20420dfffff,7,0
881e204319fffff,1,1
881e20475bfffff,7,0


In [44]:
from srai.plotting import plot_numeric_data
plot_numeric_data(regions, "amenity_bicycle_parking", wide_embedding)

In [31]:
query = {'rok': wypadki['ROK'], 'geometry': wypadki['geometry']}
features_y = gpd.GeoDataFrame(
    query,
    crs=WGS84_CRS,
    index=gpd.pd.Index(name=FEATURES_INDEX, data=[i for i in range(len(wypadki))]),
)
regions = regionalizer.transform(area)
joiner = IntersectionJoiner()
joint2 = joiner.transform(regions, features_y, return_geom=True)

/home/gry-mar/osm-proj/gradient/env/lib/python3.8/site-packages/h3ronpy/arrow/vector.py:147: DeprecationWarning: The all_intersecting parameter is deprecated and will be removed. Use containment_mode instead.
  return vector.wkb_to_cells(


In [32]:
joint2

geometry
region_id       feature_id                           
881e204053fffff 0           POINT (16.96645 51.13083)
                492         POINT (16.96645 51.13083)
                493         POINT (16.96645 51.13083)
                618         POINT (16.97336 51.13242)
                619         POINT (16.97336 51.13242)
...                                               ...
881e2042ddfffff 4074        POINT (17.02253 51.17475)
881e204205fffff 4084        POINT (16.91142 51.15944)
881e2042edfffff 4088        POINT (16.95961 51.15694)
881e2051bdfffff 4161        POINT (16.84203 51.17247)
881e2055b7fffff 4185        POINT (17.10208 51.16364)

[4538 rows x 1 columns]

In [33]:
type(joint2)

geopandas.geodataframe.GeoDataFrame